In [ ]:
import pandas as pd
import re

In [ ]:
leagues_prefixs = ["SA1", "SA2L"]
clubs_list = []
champions_list = []

In [ ]:
def attendances_tranform():
  df_f = pd.read_csv("./RAW/ArabicSoccer/Attendances/Arabic_Attendances_2007_2023.csv")

  df = df_f[df_f['#'] >= 1]
  df = df[["Stadium", "Capacity", "Spectators", "Average", "Matches", "sold out", "Club", "Year"]]

  pattern = "\B[ANH][Llab][-]?[a-zA-Z]+(?:-[a-zA-Z]+)?\s?[SFClub]{1,4}"
    
  for i in range(len(df)):
      if "Al-Ansar" in df.iloc[i, 0]:
        df.iloc[i, 6] = "Al-Ansar"
        df.iloc[i, 0] = "Prince Mohammed bin Abdul Aziz Stadium"
      elif "Ohod Club" in df.iloc[i, 0]:
        df.iloc[i, 6] = "Ohod Club"
        df.iloc[i, 0] = "Prince Mohammed bin Abdul Aziz Stadium" 
      elif "Damac FC" in df.iloc[i, 0]:
        df.iloc[i, 6] = "Damac FC"
        df.iloc[i, 0] = "Damac Club Stadium"
      else:
        df.iloc[i, 6] = re.findall(pattern=pattern, string=df.iloc[i, 0])[0]
        df.iloc[i, 0] = re.sub(pattern=pattern, string=df.iloc[i, 0], repl="")

  df = df.reset_index()
  df["index"] = df.index + 1
  df['Matches'] = pd.to_numeric(df['Matches'], downcast='integer', errors='coerce')
  df = df.rename(columns={"index": "id"})
  return df

In [ ]:
def champions_list_transform(league):
  for i in range(len(league)):
    df = pd.read_csv(f"./RAW/ArabicSoccer/Champions/{league[i]}_Champions.csv")
    df = df[["Season","Club", "League"]]
    df.rename(columns={"Club": "Clubs"}, inplace=True)
    if i == 2:
      df = df[:15]
      champions_list.append(df)
    else:
      df = df[:16]
      champions_list.append(df)

In [ ]:
def teams_transform():
  for y in range(len(leagues_prefixs)):
    df = pd.read_csv(f"./RAW/ArabicSoccer/Teams/{leagues_prefixs[y]}_teams_2023.csv")
    df = df[["Club.1", "name","Squad", "ø age", "Foreigners", "ø market value", "League"]]
    df.rename(columns={"Club.1": "Clubs", "name": "Players", "Squad":"avg_age", "ø age":"Foreigners", "Foreigners": "market_value(€)", "ø market value": "total_market_value(€)"}, inplace=True)

    for i, row in df.iterrows():
      df.iloc[i, 4] = re.sub(pattern='[€mkK]', string=row["market_value(€)"], repl="")
      df.iloc[i, 5] = re.sub(pattern='[€mkK]', string=row["total_market_value(€)"], repl="")
      
    df = df[:-1]
    df["id"] = df.index + 1
    df['"Foreigners"'] = pd.to_numeric(df["Foreigners"], downcast='integer', errors='coerce')
    df = df[["id", "Clubs","Players", "avg_age", "Foreigners", "market_value(€)", "total_market_value(€)", "League"]]
    clubs_list.append(df)

    if y == 2:
      return 


In [ ]:
df_attendances = attendances_tranform()
teams_transform()
df_teams = pd.concat(clubs_list) 
champions_list_transform(leagues_prefixs)
df_champions = pd.concat(champions_list)

In [ ]:
df_attendances.to_csv(f"./TRUSTED/ArabicSoccer/Attendances/Attendances_SPL_2007_2023.csv", encoding='utf-8', index=False)
df_teams.to_csv(f"./TRUSTED/ArabicSoccer/Teams/Teams_Arabia.csv", encoding='utf-8', index=False)
df_champions.to_csv(f"./TRUSTED/ArabicSoccer/Champions/Champions_Arabia_2007_2023.csv", encoding='utf-8', index=False)